In [1]:
import os
import json
import pickle

from bisect import bisect_left, bisect_right
from datetime import datetime, timedelta
from collections import defaultdict, Counter

import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

import matplotlib.pyplot as plt
import seaborn as sns

# GPU hack if you need
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [208]:
model_type = 'linear_extra_features'

In [3]:
def count_plot(df, x, title='', figsize=(18, 6), top=None, **kwargs):
    plt.figure(figsize=(18, 6))

    val_cnt = df[x].value_counts()
    val_cnt = val_cnt.iloc[:top] if top else val_cnt

    sns.countplot(data=df, x=x, order=val_cnt.index, **kwargs)
    plt.xticks(rotation=90, fontsize=12)
    if title:
        plt.title(title, fontsize=16)
    else:
        plt.title(f'{x.capitalize()} count', fontsize=16)
    plt.plot()
    

def calculate_product_popularity():
    product_popularity = dict()

    for df in prod_df_generator():
        df = df[df.product_id != 0]
        for product_id, cnt in df.product_id.value_counts().items():
            if product_id in product_popularity:
                product_popularity[product_id] += cnt
            else:
                product_popularity[product_id] = cnt
                
    total_cnt = sum(v for v in product_popularity.values())
    product_popularity = {k: v/total_cnt for k, v in product_popularity.items()}
    
    return product_popularity


def calcualte_user_orderes():
    user_orderes = dict()

    for df in prod_df_generator():
        for order_id, user_id in df.groupby(['order_id', 'user_id']).size().index:
            if user_id in user_orderes:
                user_orderes[user_id] += 1
            else:
                user_orderes[user_id] = 1
                
    return user_orderes


def create_user_ids():

    user_ids = set()

    for prod_df in prod_df_generator(usecols=['user_id', 'product_id']):
        prod_df = prod_df[prod_df.product_id != 0]
        prod_df.user_id = prod_df.user_id
        user_ids = user_ids.union(prod_df.user_id.unique())

    return [int(v) for v in user_ids]


def date_parser_users(d):
    try:
        d = datetime.strptime(d, '%Y-%m-%d')
    except ValueError:
        try:
            d = datetime.strptime(d, '%d.%m.%Y')
        except:
            d = datetime.strptime(d, '%d.%m')

    return d

In [4]:
PATH_TO_DATA = 'data'

PATH_TO_PRODUCTS = f'{PATH_TO_DATA}/sbermarket_tab_2_1/'
PATH_TO_ORDERS = f'{PATH_TO_DATA}/kaggle_tab_1345/tab_1_orders.csv'
PATH_TO_CATS = f'{PATH_TO_DATA}/kaggle_tab_1345/tab_3_categories.csv'
PATH_TO_PROD_PROP = f'{PATH_TO_DATA}/kaggle_tab_1345/tab_5_product_properties.csv'
PATH_TO_USERS = f'{PATH_TO_DATA}/kaggle_tab_1345/tab_4_user_profiles.csv'
PATH_TO_CITIES = f'{PATH_TO_DATA}/tab_6_city.csv'
PATH_TO_SMPL_SUBM = f'{PATH_TO_DATA}/sample_submission.csv'

PRODUCT_TABS = os.listdir(PATH_TO_PRODUCTS)

PATH_TO_PREPROC_DATA = 'preprocessed_data'
os.makedirs(PATH_TO_PREPROC_DATA, exist_ok=True)

In [5]:
UNK_TOKEN = '<UNK>'
UNK_DATE = '0000-00-00 <UNK>'
RANDOM_STATE = 42

DO_DUMP_FEATURES = False

TOP_PRODUCTS = 10000

## Read data

### Orders data

In [127]:
retailer_threshold = 1e-3

orders_df = pd.read_csv(PATH_TO_ORDERS, index_col='order_id')
orders_df = orders_df[~orders_df.index.duplicated(keep='first')]

retailer_popularity = dict((orders_df.retailer.value_counts()/len(orders_df)))

most_popular_retailers = {k: v for k, v in retailer_popularity.items() if v > retailer_threshold}
orders_df.retailer = orders_df.retailer.apply(lambda x: x if x in most_popular_retailers else UNK_TOKEN)

most_popular_platforms = ['app', 'web']
orders_df.platform = orders_df.platform.apply(lambda x: x if x in most_popular_platforms else UNK_TOKEN)
orders_df.head()

/home/a_belov/anaconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,user_id,order_created_time,retailer,store_id,platform
order_id,,,,,
17431000,72,2020-09-26 10:48:57,METRO,21,app
9718154,83,2020-05-08 09:46:18,METRO,87,web
10056850,142,2020-05-14 15:06:03,METRO,320,app
15952443,187,2020-09-01 17:34:00,ВкусВилл,533,app
10409918,224,2020-05-20 06:32:50,Ашан,183,web


## Users data

In [50]:
current_year = 2020
min_age, max_age = 13, 65

In [149]:
users_df = pd.read_csv(PATH_TO_USERS, index_col='user_id')
users_df = users_df[~users_df.index.duplicated(keep='first')]

users_df.gender = users_df.gender.fillna(UNK_TOKEN)

users_df['age'] = users_df.bdate.apply(lambda x: 
                                       0 if pd.isna(x) else 
                                       current_year - date_parser_users(x).year
                                      )

users_df.age[(users_df.age < min_age) | (users_df.age > max_age)] = 0

users_df.head()

/home/a_belov/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


,gender,bdate,age
user_id,,,
2224890,<UNK>,NaN,0
1683001,male,1987-10-11,33
2102480,<UNK>,NaN,0
2224895,<UNK>,NaN,0
930197,<UNK>,NaN,0


## Cities data

In [52]:
cities_df = pd.read_csv(PATH_TO_CITIES, index_col='store_id')
cities_df.head()

,city_name
store_id,
14,Москва
21,Москва
1,Москва
4,Москва
7,Москва


## Mapping

In [53]:
def create_mapping(values):
    mapping = {UNK_TOKEN: 0}
    return update_mapping(mapping, values)


def update_mapping(mapping, values):
    for v in values:
        v = str(v)
        if v != UNK_TOKEN and v not in mapping:
            mapping[v] = len(mapping)

    return mapping


def prod_df_generator(usecols=None):
    for tab in tqdm(PRODUCT_TABS):
        if usecols:
            yield pd.read_csv(os.path.join(PATH_TO_PRODUCTS, tab), usecols=usecols)
        else:
            yield pd.read_csv(os.path.join(PATH_TO_PRODUCTS, tab))
            

int_or_unk = lambda x: UNK_TOKEN if pd.isna(x) else int(x)


def calculate_top_products():
    prod_cnt = Counter()

    for prod_df in prod_df_generator(['product_id']):

        prod_df = prod_df[prod_df.product_id != 0]
        prod_cnt.update(prod_df.product_id.values)

    prod_cnt_df = pd.DataFrame.from_dict(prod_cnt, orient='index', columns=['product_cnt'])

    prod_cnt_df = prod_cnt_df.sort_values(by='product_cnt', ascending=False)

    return prod_cnt_df


def select_top_product(mappings, top_products):
    prod_mapping = {}
    i = 1
    for k in sorted(mappings['product_id'].keys()):
        if k == UNK_TOKEN:
            prod_mapping[k] = 0
        elif int(k) in top_products:
            prod_mapping[k] = i
            i += 1
        else:
            prod_mapping[k] = 0

    return prod_mapping

In [54]:
if DO_DUMP_FEATURES or True:
    prod_cnt_df = calculate_top_products()
    top_products = prod_cnt_df.index[:TOP_PRODUCTS].values

In [55]:
feature_cols = [('retailer', orders_df, 'orders'),
                ('platform', orders_df, 'orders'),
                ('store_id', orders_df, 'orders'),
                ('gender', users_df, 'users'),
                ('city_name', cities_df, 'cities')
               ]

feature_cols_prod = ['brand_name', 'product_id', 'master_category_id', 'parent_category_id']

if DO_DUMP_FEATURES or True:
    mappings = defaultdict(dict)

    for col, df, pref in tqdm(feature_cols):
        col_values = df[col].astype(str)

        mappings[f'{pref}_{col}'] = create_mapping(col_values.unique())

    for col in feature_cols_prod:
        mappings[col] = create_mapping([])

    for prod_df in prod_df_generator(usecols=feature_cols_prod):
        prod_df = prod_df[prod_df.product_id != 0]

        brand_values = prod_df[feature_cols_prod[0]].fillna(UNK_TOKEN)
        product_values = prod_df[feature_cols_prod[1]].fillna(UNK_TOKEN)
        master_values = prod_df[feature_cols_prod[2]].apply(int_or_unk)
        parent_values = prod_df[feature_cols_prod[3]].apply(int_or_unk)

        mappings[feature_cols_prod[0]] = update_mapping(mappings[feature_cols_prod[0]], brand_values.unique())
        mappings[feature_cols_prod[1]] = update_mapping(mappings[feature_cols_prod[1]], product_values.unique())
        mappings[feature_cols_prod[2]] = update_mapping(mappings[feature_cols_prod[2]], master_values.unique())
        mappings[feature_cols_prod[3]] = update_mapping(mappings[feature_cols_prod[3]], parent_values.unique())

    mappings['product_id'] = select_top_product(mappings, top_products)    
    
    with open(f'{PATH_TO_PREPROC_DATA}/mappings.json', 'w') as f:
        json.dump(mappings, f)

In [56]:
# load mappings
with open(f'{PATH_TO_PREPROC_DATA}/mappings.json', 'r') as f:
     mappings = json.load(f)

## Data preparing

In [11]:
if DO_DUMP_FEATURES:
    order_dates = {int(k):v for k, v in orders_df.order_created_time.items()}
    
    with open(f'{PATH_TO_PREPROC_DATA}/order_dates.json', 'w') as f:
        json.dump(order_dates, f)

with open(f'{PATH_TO_PREPROC_DATA}/order_dates.json', 'r') as f:
     order_dates = json.load(f)

In [12]:
feature_cols = ['user_id',
                'order_id',
                'product_id',
                'price',
                'quantity',
                'discount',
                'brand_name',
                'master_category_id',
                'parent_category_id'
               ]

if DO_DUMP_FEATURES:
    user_dates = defaultdict(list)
    user_prices = defaultdict(list)
    user_quantities = defaultdict(list)
    user_discounts = defaultdict(list)
    user_brand_name = defaultdict(list)
    user_master_category_ids = defaultdict(list)
    user_parent_category_ids = defaultdict(list)
    user_order_ids = defaultdict(list)
    user_product_ids = defaultdict(list)

    for prod_df in prod_df_generator(feature_cols):
        prod_df = prod_df[prod_df.product_id != 0]

        prod_df['price'] = prod_df['price'].fillna(0)
        prod_df['quantity'] = prod_df['quantity'].fillna(1)
        prod_df['discount'] = prod_df['discount'].fillna(0)
        prod_df['brand_name'] = prod_df['brand_name'].fillna(UNK_TOKEN)
        prod_df['master_category_id'] =  prod_df['master_category_id'].apply(int_or_unk).astype(str)
        prod_df['parent_category_id'] = prod_df['parent_category_id'].apply(int_or_unk).astype(str)
        prod_df['product_id'] = prod_df['product_id'].astype(str)

        for i, row in prod_df.iterrows():
            user_dates[row.user_id].append(order_dates.get(str(row.order_id), UNK_DATE + f' {row.order_id}'))
            
            user_prices[row.user_id].append(row.price)
            user_quantities[row.user_id].append(row.quantity)
            user_discounts[row.user_id].append(max(0, row.discount))
            user_brand_name[row.user_id].append(mappings['brand_name'][row.brand_name])

            user_master_category_ids[row.user_id].append(mappings['master_category_id'][row.master_category_id])
            user_parent_category_ids[row.user_id].append(mappings['parent_category_id'][row.parent_category_id])
            user_order_ids[row.user_id].append(row.order_id)
            user_product_ids[row.user_id].append(mappings['product_id'][row.product_id])


    pickle.dump(user_dates, open(f'{PATH_TO_PREPROC_DATA}/user_dates.pkl', 'wb'))
    pickle.dump(user_prices, open(f'{PATH_TO_PREPROC_DATA}/user_prices.pkl', 'wb'))
    pickle.dump(user_discounts, open(f'{PATH_TO_PREPROC_DATA}/user_discounts.pkl', 'wb'))
    pickle.dump(user_quantities, open(f'{PATH_TO_PREPROC_DATA}/user_quantities.pkl', 'wb'))
    pickle.dump(user_brand_name, open(f'{PATH_TO_PREPROC_DATA}/user_brand_name.pkl', 'wb'))
    pickle.dump(user_master_category_ids, open(f'{PATH_TO_PREPROC_DATA}/user_master_category_ids.pkl', 'wb'))
    pickle.dump(user_parent_category_ids, open(f'{PATH_TO_PREPROC_DATA}/user_parent_category_ids.pkl', 'wb'))
    pickle.dump(user_order_ids, open(f'{PATH_TO_PREPROC_DATA}/user_order_ids.pkl', 'wb'))
    pickle.dump(user_product_ids, open(f'{PATH_TO_PREPROC_DATA}/user_product_ids.pkl', 'wb'))

In [58]:
%%time

# load client data
user_dates = pickle.load(open(f'{PATH_TO_PREPROC_DATA}/user_dates.pkl', 'rb'))
user_prices = pickle.load(open(f'{PATH_TO_PREPROC_DATA}/user_prices.pkl', 'rb'))
user_discounts = pickle.load(open(f'{PATH_TO_PREPROC_DATA}/user_discounts.pkl', 'rb'))
user_quantities = pickle.load(open(f'{PATH_TO_PREPROC_DATA}/user_quantities.pkl', 'rb'))
user_brand_name = pickle.load(open(f'{PATH_TO_PREPROC_DATA}/user_brand_name.pkl', 'rb'))
user_master_category_ids = pickle.load(open(f'{PATH_TO_PREPROC_DATA}/user_master_category_ids.pkl', 'rb'))
user_parent_category_ids = pickle.load(open(f'{PATH_TO_PREPROC_DATA}/user_parent_category_ids.pkl', 'rb'))
user_order_ids = pickle.load(open(f'{PATH_TO_PREPROC_DATA}/user_order_ids.pkl', 'rb'))
user_product_ids = pickle.load(open(f'{PATH_TO_PREPROC_DATA}/user_product_ids.pkl', 'rb'))

user_ids = json.load(open(f'{PATH_TO_PREPROC_DATA}/user_ids.json'))

CPU times: user 1min 2s, sys: 1.18 s, total: 1min 3s
Wall time: 1min 3s


In [59]:
for user_id in tqdm(user_ids):
    args = np.argsort(user_dates[user_id])

    user_prices[user_id] = np.array(user_prices[user_id])[args]
    user_discounts[user_id] = np.array(user_discounts[user_id])[args]
    user_quantities[user_id] = np.array(user_quantities[user_id])[args]
    user_brand_name[user_id] = np.array(user_brand_name[user_id])[args]
    user_master_category_ids[user_id] = np.array(user_master_category_ids[user_id])[args]
    user_parent_category_ids[user_id] = np.array(user_parent_category_ids[user_id])[args]
    user_order_ids[user_id] = np.array(user_order_ids[user_id])[args]
    user_product_ids[user_id] = np.array(user_product_ids[user_id])[args]

    user_dates[user_id] = np.array(user_dates[user_id])[args]

## Train val split

In [60]:
from sklearn.model_selection import train_test_split

train_user_id, valid_user_id = train_test_split(user_ids, train_size=0.8, random_state=RANDOM_STATE)

print(f'Train: {len(train_user_id)} Val: {len(valid_user_id)}')

Train: 525944 Val: 131487


In [152]:
def get_user_data(user_id):
    user = dict()

    if user_id in users_df.index:
        user['age'] = users_df.loc[user_id].age
        user['gender'] = mappings['users_gender'][users_df.loc[user_id].gender]
    else:
        user['age'] = 0
        user['gender'] = mappings['users_gender'][UNK_TOKEN]

    return user


def get_order_data(order_id):
    order = dict()

    if order_id in orders_df.index:
        store_id = orders_df.loc[order_id].store_id
        city_name = cities_df.loc[store_id].city_name
        platform = orders_df.loc[order_id].platform
        retailer = orders_df.loc[order_id].retailer
        
        order['platform'] = mappings['orders_platform'][platform]
        order['retailer'] = mappings['orders_retailer'][retailer]
        order['city'] = mappings['cities_city_name'][city_name]
    else:
        order['platform'] = mappings['orders_platform'][UNK_TOKEN]
        order['retailer'] = mappings['orders_retailer'][UNK_TOKEN]
        order['city'] = mappings['cities_city_name'][UNK_TOKEN]
    
    return order

In [153]:
def prepare_data(user_ids, is_submission=False):
    data_prices = []
    data_discounts = []
    data_quantity = []
    data_brand_name = []
    data_master_category = []
    data_parent_category = []
    data_product = []
    data_extra = []

    data_labels = []

    for user_id in tqdm(user_ids):
        date_series = user_dates[user_id]

        price_series = user_prices[user_id]
        discount_series = user_discounts[user_id]
        quantity_series = user_quantities[user_id]
        brand_name_series = user_brand_name[user_id]
        master_category_series = user_master_category_ids[user_id]
        parent_category_series = user_parent_category_ids[user_id]

        order_id_series = user_order_ids[user_id]
        product_series = user_product_ids[user_id]

        user = get_user_data(user_id)

        if is_submission:
            date_range = [None]
        else:
            date_range = np.sort(np.unique(date_series))

        for date_end in date_range:

            if is_submission:
                l, r = len(date_series), len(date_series)
            else:
                l, r = (bisect_left(date_series, date_end),
                        bisect_right(date_series, date_end))

            history_price = price_series[:l]
            history_discount = discount_series[:l]
            history_quantity = quantity_series[:l]
            history_brand_name = brand_name_series[:l]
            history_master_category = master_category_series[:l]
            history_parent_category = parent_category_series[:l]
            history_product = product_series[:l]

            predict_product = product_series[l:r]

            if (len(predict_product) > 0) and l or is_submission:
                data_prices.append(history_price)
                data_discounts.append(history_discount)
                data_quantity.append(history_quantity)
                data_brand_name.append(history_brand_name)
                data_master_category.append(history_master_category)
                data_parent_category.append(history_parent_category)
                data_product.append(history_product)

                data_labels.append(predict_product)
            
                order_id = order_id_series[:l][-1]

                order = get_order_data(order_id)
                data_extra.append({**user, **order})

    return (data_prices, data_discounts, data_quantity, data_brand_name, 
            data_master_category, data_parent_category, data_product, data_extra, data_labels)

In [154]:
(train_prices, train_discounts, train_quantity, train_brand, train_master_category,
 train_parent_category, train_product, train_extra, train_labels) = prepare_data(user_ids)

(valid_prices, valid_discounts, valid_quantity, valid_brand, valid_master_category,
 valid_parent_category, valid_product, valid_extra, valid_labels) = prepare_data(valid_user_id)

## PyTorch loaders

In [155]:
import torch
from torch.utils.data import Dataset, DataLoader

In [167]:
PRODUCT_NCLASSES = len(np.unique(list(mappings['product_id'].values())))
MASTER_CAT_NCLASSES = len(mappings['master_category_id'])
PARENT_CAT_NCLASSES = len(mappings['parent_category_id'])
BRAND_NCLASSES = len(mappings['brand_name'])
PLATFORM_NCLASSES = len(mappings['orders_platform'])
GENDER_NCLASSES = len(mappings['users_gender'])
RETAILER_NCLASSES = len(mappings['orders_retailer'])
CITY_NCLASSES = len(mappings['cities_city_name'])

PADDING_LEN = 300

In [157]:
class ProductHistoryDataset(Dataset):
    def __init__(self, data_prices, data_discounts, data_quantity, data_brand_name, data_master_category,
                 data_parent_category, data_product, data_extra, labels=None, is_submission=False):
        super().__init__()

        self.data_prices = data_prices
        self.data_discounts = data_discounts
        self.data_quantity = data_quantity

        self.data_brand_name = data_brand_name
        self.data_master_category = data_master_category
        self.data_parent_category = data_parent_category
        self.data_product = data_product
        self.data_extra = data_extra

        self.labels = labels
        self.is_submission = is_submission
        
        self.cat_feature_names = ['brand', 'master_category', 'parent_category', 'product']
        self.cat_features = [self.data_brand_name, self.data_master_category,
                             self.data_parent_category, self.data_product]

    def __len__(self):
        return len(self.data_product)

    def __getitem__(self, idx):
        targets = np.zeros(PRODUCT_NCLASSES - 1, dtype=np.float32)
        if not self.is_submission:
            targets[self.labels[idx] - 1] = 1.

        item = {'features': {}, 'targets': targets}

        data_prices = np.array(self.data_prices[idx][-PADDING_LEN:])
        data_prices = np.vectorize(lambda s: np.log(1 + s))(data_prices)

        data_discounts = np.array(self.data_discounts[idx][-PADDING_LEN:])
        data_discounts = np.vectorize(lambda s: np.log(1 + max(0., s)))(data_discounts) 
        data_quantity = np.array(self.data_quantity[idx][-PADDING_LEN:])
        
    
        values_len = data_prices.shape[0]
        pad = np.zeros(PADDING_LEN - values_len, dtype=np.float32)
        
        data_prices = np.append(data_prices, pad)
        data_discounts = np.append(data_discounts, pad)
        data_quantity = np.append(data_quantity, pad)

        item['features']['price'] = torch.from_numpy(data_prices).float()
        item['features']['discounts'] = torch.from_numpy(data_discounts).float()
        item['features']['quantity'] = torch.from_numpy(data_quantity).float()
        
        for k, v in self.data_extra[idx].items():
            item['features'][k] = v

        for feature_name, feature_values in zip(self.cat_feature_names, [f[idx] for f in self.cat_features]):

            feature_values = np.append(np.array(feature_values[-PADDING_LEN:]), pad).astype(np.int64)
            mask = np.append(np.ones(values_len, dtype=np.float32), pad)

            item['features'][feature_name] = torch.from_numpy(feature_values).long()
            item['features'][f'{feature_name}_mask'] = torch.from_numpy(mask).float()

        return item

In [158]:
train_dataset = ProductHistoryDataset(train_prices, train_discounts, train_quantity, 
                                      train_brand, train_master_category, train_parent_category, 
                                      train_product, train_extra, train_labels)
valid_dataset = ProductHistoryDataset(valid_prices, valid_discounts, valid_quantity, 
                                      valid_brand, valid_master_category, valid_parent_category,
                                      valid_product, valid_extra, valid_labels)

In [159]:
train_loader = DataLoader(
    train_dataset, batch_size=64, shuffle=True, num_workers=4
)
valid_loader = DataLoader(
    valid_dataset, batch_size=64, shuffle=False, num_workers=4
)

In [160]:
batch = next(iter(train_loader))
torch.cat([batch['features']['price'][:2].unsqueeze(-1), 
           batch['features']['brand'][:2].unsqueeze(-1)], dim=-1).shape

torch.Size([2, 300, 2])

## Model

In [164]:
import torch.nn as nn

from collections import OrderedDict

In [168]:
PRODUCT_NCLASSES, MASTER_CAT_NCLASSES, PARENT_CAT_NCLASSES, BRAND_NCLASSES, \
PLATFORM_NCLASSES, GENDER_NCLASSES, RETAILER_NCLASSES, CITY_NCLASSES

(10001, 612, 120, 7612, 3, 3, 7, 102)

In [173]:
params = {'product_emb_dim': 300,
          'master_cat_emb_dim': 32,
          'parent_cat_emb_dim': 16,
          'brand_emb_dim': 32,
          
          'platform_emb_dim': 2,
          'retailer_emb_dim': 4,
          'cuty_emb_dim': 4,
          'gender_emb_dim': 2,

          'transformer_nhead': 2,
          'transformer_dim_feedforward': 300,

          'transformer_dropout': 0.1,
          'dense_unit': 256,
          'num_layers': 4,
         }

In [210]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()

        self.product_embedding = nn.Embedding(PRODUCT_NCLASSES, params['product_emb_dim'])
        self.master_cat_embedding = nn.Embedding(MASTER_CAT_NCLASSES, params['master_cat_emb_dim'])
        self.parent_cat_embedding = nn.Embedding(PARENT_CAT_NCLASSES, params['parent_cat_emb_dim'])
        self.brend_embedding = nn.Embedding(BRAND_NCLASSES, params['brand_emb_dim'])

        self.platform_embedding = nn.Embedding(PLATFORM_NCLASSES, params['platform_emb_dim'])
        self.retailer_embedding = nn.Embedding(RETAILER_NCLASSES, params['retailer_emb_dim'])
        self.city_embedding = nn.Embedding(CITY_NCLASSES, params['cuty_emb_dim'])
        self.gender_embedding = nn.Embedding(GENDER_NCLASSES, params['gender_emb_dim'])

        embedding_size = (params['product_emb_dim'] + params['master_cat_emb_dim'] +
                          params['parent_cat_emb_dim'] + params['brand_emb_dim'] +
                          params['platform_emb_dim'] + params['retailer_emb_dim'] +
                          params['cuty_emb_dim'] + params['gender_emb_dim'] +
                          4
                          )

        transformer_blocks = [(f'transformer_block_{i}', 
                               nn.TransformerEncoderLayer(d_model=embedding_size,
                                                          nhead=params['transformer_nhead'],
                                                          dim_feedforward=params['transformer_dim_feedforward'],
                                                          dropout=params['transformer_dropout']
                                                         )
                              ) for i in range(params['num_layers'])]
 
        self.transformer_encoder = nn.Sequential(OrderedDict(transformer_blocks))

        self.body = nn.Sequential(
            nn.Linear(in_features=embedding_size, out_features=512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            
            nn.Linear(in_features=512, out_features=512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            
            nn.Linear(in_features=512, out_features=512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.3)
                                 )

        self.scorer = nn.Linear(in_features=512, out_features=PRODUCT_NCLASSES - 1)

    def forward(self, features):

        product_emb = self.product_embedding(features['product'])
        master_cat_emb = self.master_cat_embedding(features['master_category'])
        parent_cat_emb = self.parent_cat_embedding(features['parent_category'])
        brand_emb = self.brend_embedding(features['brand'])

        product_emb = product_emb * features['product_mask'].unsqueeze(-1)
        master_cat_emb = master_cat_emb * features['master_category_mask'].unsqueeze(-1)
        parent_cat_emb = parent_cat_emb * features['parent_category_mask'].unsqueeze(-1)
        brand_emb = brand_emb * features['brand_mask'].unsqueeze(-1)

        platform_emb = self.platform_embedding(features['platform']).unsqueeze(1).repeat(1, PADDING_LEN, 1)
        retailer_emb = self.retailer_embedding(features['retailer']).unsqueeze(1).repeat(1, PADDING_LEN, 1)
        city_emb = self.city_embedding(features['city']).unsqueeze(1).repeat(1, PADDING_LEN, 1)
        gender_emb = self.gender_embedding(features['gender']).unsqueeze(1).repeat(1, PADDING_LEN, 1)
        
        age = features['quantity'].unsqueeze(-1)#.repeat(1, PADDING_LEN, 1)
        
        #print(age.shape, brand_emb.shape, platform_emb.shape)
        embeddings = torch.cat((product_emb, master_cat_emb, parent_cat_emb, brand_emb, 
                                platform_emb, retailer_emb, city_emb, gender_emb,
                                features['price'].unsqueeze(-1), features['discounts'].unsqueeze(-1),
                                features['quantity'].unsqueeze(-1),  age
                               ), dim=-1)

        transformer_output = self.transformer_encoder(embeddings)

        pooling = torch.mean(transformer_output, dim=1)
        #print(pooling.shape)
        
        body = torch.tanh(self.body(pooling))
        merch_logits = self.scorer(body)

        return merch_logits

### One-batch-check

In [211]:
model = Model()
criterion = nn.BCEWithLogitsLoss()
batch = next(iter(train_loader))
output = model(batch['features'])
loss = criterion(output, batch['targets'])
print(loss)

print('Model parameters:', sum(p.numel() for p in model.parameters() if p.requires_grad))

tensor(0.7003, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
Model parameters: 12602396


In [212]:
from catalyst import dl, utils
from catalyst.utils import metrics

In [213]:
from typing import List, Optional, Sequence, Tuple, Union

import numpy as np
import torch
from catalyst.utils.metrics.functional import preprocess_multi_label_metrics
from catalyst.utils.torch import get_activation_fn


def multi_label_metrics(
    outputs: torch.Tensor,
    targets: torch.Tensor,
    threshold: Union[float, torch.Tensor],
    activation: Optional[str] = None,
    eps: float = 1e-7,
) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor, torch.Tensor]:
    """
    Computes multi-label precision for the specified activation and threshold.

    Args:
        outputs (torch.Tensor): NxK tensor that for each of the N examples
            indicates the probability of the example belonging to each of
            the K classes, according to the model.
        targets (torch.Tensor): binary NxK tensort that encodes which of the K
            classes are associated with the N-th input
            (eg: a row [0, 1, 0, 1] indicates that the example is
            associated with classes 2 and 4)
        threshold (float): threshold for for model output
        activation (str): activation to use for model output
        eps (float): epsilon to avoid zero division
    
    Extended version of 
        https://github.com/catalyst-team/catalyst/blob/master/catalyst/utils/metrics/accuracy.py#L58

    Returns:
        computed multi-label metrics
    """
    outputs, targets, _ = preprocess_multi_label_metrics(
        outputs=outputs, targets=targets
    )
    activation_fn = get_activation_fn(activation)
    outputs = activation_fn(outputs)

    outputs = (outputs > threshold).long()

    accuracy = (targets.long() == outputs.long()).sum().float() / np.prod(
        targets.shape
    )

    intersection = (outputs.long() * targets.long()).sum(axis=1).float()
    num_predicted = outputs.long().sum(axis=1).float()
    num_relevant = targets.long().sum(axis=1).float()
    union = num_predicted + num_relevant

    # Precision = ({predicted items} && {relevant items}) / {predicted items}
    precision = intersection / (num_predicted + eps * (num_predicted == 0))
    # Recall = ({predicted items} && {relevant items}) / {relevant items}
    recall = intersection / (num_relevant + eps * (num_relevant == 0))
    # IoU = ({predicted items} && {relevant items}) / ({predicted items} || {relevant items})
    iou = (intersection + eps * (union == 0)) / (union - intersection + eps)

    return accuracy, precision.mean(), recall.mean(), iou.mean()


def precision_at_k(
    actual: torch.Tensor, 
    predicted: torch.Tensor, 
    k: int,
):
    """
    Computes precision at cutoff k for one sample

    Args:
       actual: (torch.Tensor): tensor of length K with predicted item_ids sorted by relevance
       predicted (torch.Tensor): binary tensor that encodes which of the K
           classes are associated with the N-th input
           (eg: a row [0, 1, 0, 1] indicates that the example is
           associated with classes 2 and 4)
       k (int): parameter k of precison@k

    Returns:
       Computed value of precision@k for given sample
    """
    p_at_k = 0.0
    for item in predicted[:k]:
        if actual[item]:
            p_at_k += 1
    p_at_k /= k

    return p_at_k


def average_precision_at_k(
    actual: torch.Tensor, 
    predicted: torch.Tensor, 
    k: int,
) -> float:
    """
    Computes average precision at cutoff k for one sample

    Args:
      actual: (torch.Tensor): tensor of length K with predicted item_ids sorted by relevance
      predicted (torch.Tensor): binary tensor that encodes which of the K
          classes are associated with the N-th input
          (eg: a row [0, 1, 0, 1] indicates that the example is
          associated with classes 2 and 4)
      k (int): parameter k of AP@k

    Returns:
        Computed value of AP@k for given sample
    """
    ap_at_k = 0.0
    for idx, item in enumerate(predicted[:k]):
        if actual[item]:
            ap_at_k += precision_at_k(actual, predicted, k=idx + 1)
    ap_at_k /= min(k, actual.sum().cpu().numpy())
    

    return ap_at_k


def mean_average_precision_at_k(
    output: torch.Tensor, target: torch.Tensor, top_k: Tuple[int, ...] = (1,)
) -> List[float]:
    """
    Computes mean_average_precision_at_k at set of cutoff parameters K

    Args:
       outputs (torch.Tensor): NxK tensor that for each of the N examples
           indicates the probability of the example belonging to each of
           the K classes, according to the model.
       targets (torch.Tensor): binary NxK tensort that encodes which of the K
           classes are associated with the N-th input
           (eg: a row [0, 1, 0, 1] indicates that the example is
           associated with classes 2 and 4)
       top_k (tuple): list of parameters k at which map@k will be computed


    Returns:
       List of computed values of map@k at each cutoff k from topk
    """
    max_k = max(top_k)
    batch_size = target.size(0)

    _, top_indices = output.topk(k=max_k, dim=1, largest=True, sorted=True)

    result = []
    for k in top_k:  # loop over k
        map_at_k = 0.0
        for actual_target, predicted_items in zip(
            target, top_indices
        ):  # loop over samples
            map_at_k += average_precision_at_k(
                actual_target, predicted_items, k
            )
        map_at_k = map_at_k / batch_size
        result.append(map_at_k)

    return result

In [214]:
class CustomRunner(dl.Runner):

    def _handle_batch(self, batch):
        features, targets = batch['features'], batch['targets']
        logits = self.model(features)
        scores = torch.sigmoid(logits)

        loss = self.criterion(logits, targets)
        accuracy, precision, recall, iou = multi_label_metrics(logits, targets, 
                                                               threshold=0.5, activation='Sigmoid'
                                                              )
        (map50, ) = mean_average_precision_at_k(scores, targets, top_k=(50,))
        batch_metrics = {'loss': loss,
                         'precision': precision,
                         'recall': recall,
                         'map50': map50,   
                       }

        self.input = {'features': features, 'targets': targets}
        self.output = {'logits': logits, 'scores': scores}
        self.batch_metrics.update(batch_metrics)

        if self.is_train_loader:
            loss.backward()
            self.optimizer.step()
            self.optimizer.zero_grad()
    
    def predict_batch(self, batch):
        # model inference step
        batch = utils.maybe_recursive_call(batch, 'to', device=self.device)
        logits = self.model(batch['features'])
        scores = torch.sigmoid(logits)
        return scores

In [215]:
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [216]:
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
print(device)

model = Model()
#model.to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

loaders = {"train": train_loader, "valid": valid_loader}

cuda:0


In [ ]:
runner = CustomRunner()

runner.train(model=model,
             criterion=criterion,
             optimizer=optimizer,
             scheduler=None,
             loaders=loaders,
             logdir=f'./logs/{model_type}',
             num_epochs=30,
             verbose=True,
             load_best_on_end=True,
             overfit=False,  #  <<<--- DO NOT FORGET TO MAKE IT ``False`` 
                             #  (``True`` uses only one batch to check pipeline correctness)
             callbacks=[
                 # https://scikit-learn.org/stable/modules/generated/sklearn.metrics.average_precision_score.html
                 # dl.AveragePrecisionCallback(input_key="targets", output_key="scores", prefix="ap"),
                 # https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html
                 # dl.AUCCallback(input_key="targets", output_key="scores", prefix="auc"),
             ],

             main_metric='iou', 
             minimize_metric=False,
            )

1/30 * Epoch (train):   2% 516/24237 [02:01<1:33:53,  4.21it/s, loss=0.013, map50=0.087, precision=0.984, recall=0.071]   

## Submission

In [ ]:
pd_sbm = pd.read_csv(PATH_TO_SMPL_SUBM, index_col='Id')
submit_ids = pd_sbm.index.values
pd_sbm.head()

In [ ]:
test_prices, *_ = prepare_data(submit_ids, is_submission=True)
submit_ids_with_data_idx = [i for i, tp in enumerate(test_prices) if len(tp)]
submit_ids_with_data = submit_ids[submit_ids_with_data_idx]

(test_prices, test_discounts, test_quantity, test_brand, 
 test_master_category, test_parent_category, test_product, test_labels) = prepare_data(submit_ids_with_data, 
                                                                                       is_submission=True)

full_dataset = ProductHistoryDataset(test_prices, test_discounts, test_quantity, 
                                     test_brand, test_master_category, test_parent_category,
                                     test_product, is_submission=True)
full_loader = DataLoader(full_dataset, batch_size=64, shuffle=False, num_workers=8, drop_last=False)

In [ ]:
predictions = []
for scores in tqdm(runner.predict_loader(loader=full_loader), total = len(full_loader)):
    _, top_indices = scores.topk(k=50, dim=1, largest=True, sorted=True)
    top_indices += 1
    predictions += top_indices.detach().cpu().tolist()

In [ ]:
# inverse mapping for merchant_type in predictions
product_inverse_mapping = {k: int(v) for v, k in mappings['product_id'].items() if v != UNK_TOKEN}

def inverse_mapping(x):
    return list(map(product_inverse_mapping.get, x))

predictions_prod_id = list(map(inverse_mapping, predictions))

In [ ]:
submission = pd_sbm.copy()

for i, p in tqdm(zip(submit_ids_with_data, predictions_prod_id)):
    submission.loc[i].Predicted = ' '.join(map(str, p))

sumb_path = f'submission_Hugs_for_Bugs_{model_type}.csv'
submission.to_csv(sumb_path, sep=',')

In [ ]:
!kaggle competitions submit test-recsys -f $sumb_path -m $model_type